In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/peia"

/content/drive/MyDrive/peia


In [ ]:
pip install onnx

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np

class DNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_hidden_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 4*hidden_size),
            nn.ReLU(),
            *[nn.Sequential(nn.Linear(4*hidden_size, 4*hidden_size), nn.ReLU(), nn.Dropout(dropout)) for _ in range(num_hidden_layers)],
            nn.Linear(4*hidden_size, hidden_size),
        )
    def forward(self, x):
        return self.layers(x)

class Score(nn.Module):
    def __init__(self, d_model):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Sigmoid(),
            nn.Linear(d_model, 1),
        )

    def forward(self, x):
        return self.layers(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model,dropout=0.1):
        super().__init__()

        self.linear = nn.Linear(d_model*d_model, d_model)
        self.l_to_score, self.s_to_score, self.d_to_score = [Score(d_model) for _ in range(3)]
        self.dropout = nn.Dropout(dropout)

    def forward(self, user_embedded, mood_embedded, genre_embedded):
        l = (user_embedded.unsqueeze(2)*genre_embedded.unsqueeze(1)).reshape(genre_embedded.shape[0], 6, -1)

        l_score = self.l_to_score(l)

        alpha = f.softmax(l_score, dim=1)
        l_att = (alpha * l).sum(dim=1)

        s = (mood_embedded.unsqueeze(2)*genre_embedded.unsqueeze(1)).reshape(genre_embedded.shape[0], 2, -1)

        s_score = self.s_to_score(s)

        beta = f.softmax(s_score,dim=1)
        s_att = (beta*s).sum(dim=1)

        ddl = self.d_to_score(l_att)

        dds = self.d_to_score(s_att)

        dl = torch.exp(ddl)/(torch.exp(ddl)+torch.exp(dds))
        ds = torch.exp(dds)/(torch.exp(ddl)+torch.exp(dds))

        out = dl * l_att + ds * s_att

        return out, dl, ds

class Embeddings(nn.Module):
    def __init__(self, d_model,dropout=0.1):
        super().__init__()
        self.linear = nn.Linear(1,d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        linear_output = self.linear(input)
        embeddings = self.dropout(linear_output)
        return embeddings

class Embedder(nn.Module):
    def __init__(self,d_model):
        super().__init__()

        self.user_embedding = Embeddings(d_model)
        self.mood_embedding = Embeddings(d_model)
        self.genre_embedding = Embeddings(d_model)

    def forward(self, user_v, mood_v, genre_v):
        user_embedded = self.user_embedding(user_v)
        mood_embedded = self.mood_embedding(mood_v)
        genre_embedded = self.genre_embedding(genre_v)

        return user_embedded, mood_embedded, genre_embedded

class TransformerClassifier(nn.Module):
    def __init__(self, d_model, user_num, mood_num, genre_num, num_hidden_layers, dropout=0.1):
        super().__init__()

        self.all_num = user_num + mood_num + genre_num

        self.embedder = Embedder(d_model)
        self.mha = MultiHeadAttention(d_model)
        self.dnn = DNN(d_model*self.all_num, d_model, num_hidden_layers)

        self.W_zd = nn.Linear(d_model, 10, bias=False)
        self.W_z0 = nn.Linear(self.all_num, 10, bias=False)
        self.W_zatt = nn.Linear(d_model, 10, bias=False)
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, user_v, mood_v, genre_v):

        user_embedded, mood_embedded, genre_embedded = self.embedder(user_v, mood_v, genre_v)

        all_embedded_values = torch.cat([user_embedded, mood_embedded, genre_embedded], dim=1)

        all_embedded_values_r = all_embedded_values.reshape(all_embedded_values.shape[0], -1)

        zd = self.dnn(all_embedded_values_r)

        z0 = torch.cat((user_v, mood_v, genre_v), dim=1)

        z0_r = z0.reshape(z0.shape[0],-1)

        zatt, rl, rs = self.mha(user_embedded,mood_embedded, genre_embedded)

        out = torch.sigmoid(self.W_zd(zd) + self.W_z0(z0_r) + self.W_zatt(zatt) + self.bias)

        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

user_v = torch.rand(1,3,1).to(device)
mood_v = torch.rand(1,1,1).to(device)
genre_v = torch.rand(1,2,1).to(device)

x = (user_v, mood_v, genre_v)
model_params = torch.load('model_parameters_label.pt', map_location=device)
model = TransformerClassifier(d_model=8, user_num=user_v.shape[1], mood_num=mood_v.shape[1], genre_num = genre_v.shape[1], num_hidden_layers=2).to(device)
model.load_state_dict(model_params)
model.eval()
torch.onnx.export(model,x, "peia_new.onnx")